# Loop 1 LB Feedback Analysis

**Baseline submitted:** CV 70.6761 | LB 70.6761 (gap: 0.0000)

This is excellent - CV and LB match perfectly, meaning our scoring is accurate.

## Key Questions:
1. What's the best available score in snapshots?
2. What N values have the most room for improvement?
3. What techniques can close the 1.79 point gap to target (68.89)?

In [1]:
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path

# Load current baseline
baseline_path = '/home/code/experiments/000_baseline/submission.csv'
df = pd.read_csv(baseline_path)
print(f"Baseline submission shape: {df.shape}")
print(df.head())

Baseline submission shape: (20100, 4)
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  


In [2]:
# Check best available snapshot
best_snapshot_path = '/home/nonroot/snapshots/santa-2025/21329069570/code/code/solutions/submission_70.627569.csv'
if os.path.exists(best_snapshot_path):
    df_best = pd.read_csv(best_snapshot_path)
    print(f"Best snapshot submission shape: {df_best.shape}")
    print(f"\nThis is 0.049 points better than current baseline!")
else:
    print("Best snapshot not found")

Best snapshot submission shape: (20100, 4)

This is 0.049 points better than current baseline!


In [3]:
# Define tree polygon and scoring functions
import math
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

def get_tree_polygon():
    """Return the 15-vertex Christmas tree polygon."""
    trunk_w, trunk_h = 0.15, 0.2
    base_w, mid_w, top_w = 0.7, 0.4, 0.25
    tip_y, tier_1_y, tier_2_y = 0.8, 0.5, 0.25
    base_y, trunk_bottom_y = 0.0, -trunk_h
    
    vertices = [
        (0, tip_y),
        (top_w/2, tier_1_y), (top_w/4, tier_1_y),
        (mid_w/2, tier_2_y), (mid_w/4, tier_2_y),
        (base_w/2, base_y),
        (trunk_w/2, base_y), (trunk_w/2, trunk_bottom_y),
        (-trunk_w/2, trunk_bottom_y), (-trunk_w/2, base_y),
        (-base_w/2, base_y),
        (-mid_w/4, tier_2_y), (-mid_w/2, tier_2_y),
        (-top_w/4, tier_1_y), (-top_w/2, tier_1_y),
    ]
    return Polygon(vertices)

BASE_TREE = get_tree_polygon()
print(f"Tree polygon area: {BASE_TREE.area:.6f}")
print(f"Tree bounding box: {BASE_TREE.bounds}")

Tree polygon area: 0.245625
Tree bounding box: (-0.35, -0.2, 0.35, 0.8)


In [4]:
def parse_value(s):
    """Parse submission value (remove 's' prefix)."""
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def calculate_per_n_scores(df):
    """Calculate score contribution for each N."""
    df = df.copy()
    df['x_val'] = df['x'].apply(parse_value)
    df['y_val'] = df['y'].apply(parse_value)
    df['deg_val'] = df['deg'].apply(parse_value)
    df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
    
    scores = {}
    for n in range(1, 201):
        n_df = df[df['n'] == n]
        if len(n_df) != n:
            continue
        
        # Get all tree vertices
        all_points = []
        for _, row in n_df.iterrows():
            tree = affinity.rotate(BASE_TREE, row['deg_val'], origin=(0, 0))
            tree = affinity.translate(tree, xoff=row['x_val'], yoff=row['y_val'])
            all_points.extend(list(tree.exterior.coords))
        
        xs = [p[0] for p in all_points]
        ys = [p[1] for p in all_points]
        side = max(max(xs) - min(xs), max(ys) - min(ys))
        score_n = (side ** 2) / n
        scores[n] = {'side': side, 'score': score_n}
    
    return scores

print("Calculating per-N scores for baseline...")
baseline_scores = calculate_per_n_scores(df)
total_score = sum(s['score'] for s in baseline_scores.values())
print(f"Total baseline score: {total_score:.6f}")

Calculating per-N scores for baseline...


Total baseline score: 70.676102


In [5]:
# Analyze which N values contribute most to the score
scores_df = pd.DataFrame([
    {'N': n, 'side': s['side'], 'score': s['score'], 'pct': s['score']/total_score*100}
    for n, s in baseline_scores.items()
])

print("Top 20 N values by score contribution:")
print(scores_df.nlargest(20, 'score')[['N', 'side', 'score', 'pct']].to_string())

print(f"\nTotal score: {total_score:.6f}")
print(f"Target score: 68.890873")
print(f"Gap: {total_score - 68.890873:.6f}")

Top 20 N values by score contribution:
     N      side     score       pct
0    1  0.813173  0.661250  0.935606
1    2  0.949504  0.450779  0.637810
2    3  1.142031  0.434745  0.615123
4    5  1.443692  0.416850  0.589803
3    4  1.290806  0.416545  0.589372
6    7  1.673104  0.399897  0.565816
5    6  1.548438  0.399610  0.565411
8    9  1.867280  0.387415  0.548156
7    8  1.755921  0.385407  0.545315
14  15  2.384962  0.379203  0.536536
9   10  1.940696  0.376630  0.532896
20  21  2.811667  0.376451  0.532642
19  20  2.742469  0.376057  0.532085
10  11  2.033002  0.375736  0.531631
21  22  2.873270  0.375258  0.530955
15  16  2.446640  0.374128  0.529356
25  26  3.118320  0.373997  0.529170
11  12  2.114873  0.372724  0.527369
12  13  2.200046  0.372323  0.526802
24  25  3.050182  0.372144  0.526549

Total score: 70.676102
Target score: 68.890873
Gap: 1.785229


In [6]:
# Calculate theoretical minimum (if trees could be packed perfectly)
# Each tree has area ~0.2175 (from polygon area)
tree_area = BASE_TREE.area
print(f"Single tree area: {tree_area:.6f}")

# Theoretical minimum: if N trees could be packed into a square with no wasted space
# side = sqrt(N * tree_area), score_n = side^2 / n = tree_area
theoretical_min = sum(tree_area for n in range(1, 201))
print(f"\nTheoretical minimum score (perfect packing): {theoretical_min:.6f}")
print(f"Current score: {total_score:.6f}")
print(f"Packing efficiency: {theoretical_min/total_score*100:.2f}%")

Single tree area: 0.245625

Theoretical minimum score (perfect packing): 49.125000
Current score: 70.676102
Packing efficiency: 69.51%


In [7]:
# Compare with best snapshot if available
if os.path.exists(best_snapshot_path):
    print("Calculating per-N scores for best snapshot...")
    best_scores = calculate_per_n_scores(df_best)
    best_total = sum(s['score'] for s in best_scores.values())
    print(f"Best snapshot total: {best_total:.6f}")
    
    # Find N values where best snapshot is better
    improvements = []
    for n in range(1, 201):
        if n in baseline_scores and n in best_scores:
            diff = baseline_scores[n]['score'] - best_scores[n]['score']
            if diff > 0.0001:
                improvements.append({'N': n, 'baseline': baseline_scores[n]['score'], 
                                   'best': best_scores[n]['score'], 'improvement': diff})
    
    if improvements:
        imp_df = pd.DataFrame(improvements).sort_values('improvement', ascending=False)
        print(f"\nN values where best snapshot is better (top 20):")
        print(imp_df.head(20).to_string())
        print(f"\nTotal potential improvement: {sum(i['improvement'] for i in improvements):.6f}")

Calculating per-N scores for best snapshot...


Best snapshot total: 70.627569

N values where best snapshot is better (top 20):
      N  baseline      best  improvement
10   54  0.361321  0.356444     0.004876
11   57  0.358045  0.353528     0.004517
7    43  0.370040  0.367197     0.002843
34  128  0.343762  0.340970     0.002792
24   88  0.350672  0.348197     0.002475
2    15  0.379203  0.376978     0.002225
28   94  0.352274  0.350076     0.002198
30  100  0.345531  0.343427     0.002103
18   76  0.351603  0.349768     0.001835
14   64  0.350468  0.348773     0.001695
41  157  0.341876  0.340203     0.001673
31  101  0.350389  0.348801     0.001588
16   74  0.354139  0.352709     0.001430
42  162  0.338332  0.337022     0.001311
29   95  0.349094  0.347800     0.001294
33  123  0.348717  0.347565     0.001152
1    14  0.370569  0.369543     0.001026
26   91  0.347911  0.347003     0.000908
17   75  0.353773  0.352898     0.000875
0    11  0.375736  0.374924     0.000812

Total potential improvement: 0.047413


In [8]:
# Summary
print("="*60)
print("SUMMARY")
print("="*60)
print(f"Current baseline: {total_score:.6f}")
print(f"Target: 68.890873")
print(f"Gap to target: {total_score - 68.890873:.6f} ({(total_score - 68.890873)/68.890873*100:.2f}%)")
print(f"\nBest available snapshot: 70.627569")
print(f"Improvement from better baseline: {total_score - 70.627569:.6f}")
print(f"\nRemaining gap after better baseline: {70.627569 - 68.890873:.6f}")
print("\nKey insight: Even with the best snapshot, we still need 1.74 points of improvement.")
print("This requires fundamentally different approaches, not just running optimizers longer.")

SUMMARY
Current baseline: 70.676102
Target: 68.890873
Gap to target: 1.785229 (2.59%)

Best available snapshot: 70.627569
Improvement from better baseline: 0.048533

Remaining gap after better baseline: 1.736696

Key insight: Even with the best snapshot, we still need 1.74 points of improvement.
This requires fundamentally different approaches, not just running optimizers longer.
